In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=5bdf2b421c559b6c18dd8d2ae2bf0e1616723ed45bf0699772e67275208d1879
  Stored in directory: /tmp/pip-ephem-wheel-cache-nyffom7v/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
from google.colab import drive
drive.mount("/content/drive") 
path = 'drive//My\\ Drive//Lambda//kickstarter//kickstarter.csv'
path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'drive//My\\ Drive//Lambda//kickstarter//kickstarter.csv'

In [ ]:
path = 'drive/My Drive/Lambda/kickstarter/kickstarter.csv'

In [ ]:
df = pd.read_csv(path, index_col=0)
df['binary_state'] = df['state'].apply(lambda state: 0 if state == 'failed' else 1)

In [ ]:
df.head(2)

,blurb,state,binary_state
0,"Using their own character, users go on educati...",failed,0
1,"MicroFly is a quadcopter packed with WiFi, 6 s...",successful,1


In [ ]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1,2),
                        max_df=.97,min_df=3,
                        tokenizer=(lambda text: [token.lemma_.strip() for token
                                                in nlp(text) if 
                                                (token.is_stop != True) & 
                                                (token.is_punct != True) & 
                                                (token.text is not ' ')]))

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(df['blurb'])

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

In [ ]:
from sklearn.linear_model import LinearRegression

# Fit on DTM
nn = LinearRegression()
nn.fit(dtm, df['binary_state'])

In [ ]:
nn.predict(tfidf.transform(["This is my example description of a love love love great product"]).todense())